### Alignment

An example of how we aligned samples for mCREATE. This aligns FASTQ files against an expected variant template, and then exports the counts and enrichment of one sample over another.

In [ ]:
import os

from pepars.fileio import fileio

from protfarm.workspace import Workspace as ws
from protfarm.workspace import Database as db
from protfarm.workspace import FASTQ_File as FASTQ_File
from protfarm.workspace import Library
from protfarm.workspace import Template
from protfarm.workspace import Alignment
from protfarm.analysis import Analysis_Set

In [ ]:
# The data path represents the location of all protein engineering sequencing experiments
DATA_PATH = ("example_data")

# Each experiment is given its own name and subdirectory in the DATA PATH
# An experiment is a group of samples, all using the same variant region
EXPERIMENT_NAME = "mCREATE"

TEMPLATE_SEQUENCE = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXGAGTGCCCAANNKNNKNNKNNKNNKNNKNNKGCACAGGCGCXXXXXXXXXXXXXXXXXXXX"
TEMPLATE_NAME = "7-mer NNK Insertion Position 56"

VARIANT_QUALITY_THRESHOLD = 30
MISMATCH_QUALITY_THRESHOLD = 20

# Currently, only Perfect_Match_Aligner is functional
ALIGNMENT_METHOD = "Perfect_Match_Aligner"

# A map of samples and the FASTQ files associated with each - this can be entered manually or read
# in from an Excel sheet
SAMPLE_FASTQ_FILES = {
    "sample_1": ["sample_1.fastq.gz"],
    "sample_2": ["sample_2.fastq.gz"]
}

In [ ]:
# Set the data and experiment path
ws.set_data_path(DATA_PATH)
ws.set_experiment(EXPERIMENT_NAME)

In [ ]:
# Seed our experiment with some FASTQ files to download
REMOTE_FILES = [
    ("https://caltech.box.com/shared/static/5a1zi1pawtn1x15tupr1pub01wqa5kfg.gz", "sample_1.fastq.gz"),
    ("https://caltech.box.com/shared/static/fvu4uq3bjuur2hufjlzit0ijt3m1ji2i.gz", "sample_2.fastq.gz")
]

for remote_file_URL, local_file_name in REMOTE_FILES:
    
    # FASTQ files go in the raw data folder
    FASTQ_file_path = ws.get_raw_data_path(local_file_name)

    # This downloads the file, but only if it doesn't already exist
    fileio.download_remote_file(remote_file_URL, FASTQ_file_path)

# Reload the workspace to get the newly added FASTQ file
ws.set_experiment(EXPERIMENT_NAME)

In [ ]:
# Create libraries and associate FASTQ files for each of them
for sample_name, FASTQ_file_names in SAMPLE_FASTQ_FILES.items():
    
    try:
        library = Library(sample_name)
    except Exception as e:
        library = db.get_library(sample_name)

    for FASTQ_file_name in FASTQ_file_names:
        library.add_file(FASTQ_file_name)

In [ ]:
try:
    template = Template(TEMPLATE_SEQUENCE, name=TEMPLATE_NAME)
except Exception:
    template = db.get_template_by_name(TEMPLATE_NAME)

In [ ]:
sample_templates = {}

for sample in db.get_samples():
    sample_templates[sample.id] = template.id

alignment_parameters = {
    "mismatch_quality_threshold": MISMATCH_QUALITY_THRESHOLD,
    "variant_sequence_quality_threshold": VARIANT_QUALITY_THRESHOLD
}

try:
    alignment = Alignment(ALIGNMENT_METHOD,
                          parameters=alignment_parameters,
                          library_templates=sample_templates)
except ValueError as e:
    print(e)
    alignment = db.get_alignment_by_parameters(ALIGNMENT_METHOD, alignment_parameters, sample_templates)

In [ ]:
# Align all will align all unaligned samples
ws.align_all(print)

In [ ]:
# Get the results of this alignment and print the alignment statistics

alignment = db.get_alignments()[0]

for sample_id in alignment.statistics:
    
    sample_name = db.get_library_by_id(sample_id).name
    
    print("Alignment statistics for '%s'" % sample_name)
    
    for key, value in alignment.statistics[sample_id].items():
    
        print("%s: %.4f" % (key, value))
        
    print("")

In [ ]:
# This exports all alignment statistics to a file alignment_statistics.csv in the export folder
ws.export_alignment_statistics()

ws.set_active_alignment(alignment)

In [ ]:
STARTING_SAMPLE_NAME = "sample_1"
ENRICHED_SAMPLE_NAME = "sample_2"

# Whether to collapse sequences that are similar (off by one nucleotide)
COLLAPSE_SIMILAR_SEQUENCES = False

# The name of the file to export
EXPORT_FILE_NAME = "example_enrichment.csv"

# The minimum count a sequence must have across all samples to be considered in analysis
COUNT_THRESHOLD = 0

In [ ]:
# Now we create an Analysis Set - this is a set of samples used for analysis
analysis_set = Analysis_Set()

analysis_set.add_sample(STARTING_SAMPLE_NAME)
analysis_set.add_sample(ENRICHED_SAMPLE_NAME)

In [ ]:
if COLLAPSE_SIMILAR_SEQUENCES:
    for sequence_library_name, sequence_library in analysis_set.get_libraries().items():
        sequence_library.collapse_sequence_counts()

In [ ]:
# This calculates the enrichment of each sample with respect to the starting sample, and exports it to a file
analysis_set.export_enrichment_specificity(
    EXPORT_FILE_NAME,
    STARTING_SAMPLE_NAME,
    libraries_to_compare_names=None,
    count_threshold=COUNT_THRESHOLD)